# Raw Download Example

## Objectives
1. <a href="#find">Find the latest latest available output from the GFS CONUS 95km</a>
1. <a href="#local">Download the latest output</a>
1. <a href="#open">Open the latest output as python file object</a>

<a name="find"></a>
Let's get the latest available output from the GFS CONUS 95km

In [1]:
from siphon.catalog import TDSCatalog
cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/'
                 'grib/NCEP/GFS/CONUS_95km/catalog.xml')

In [2]:
latest_dataset = cat.latest

In [3]:
latest_dataset

GFS_CONUS_95km_20180109_1200.grib2

<a name="local"></a>
We can ask Siphon to download the file locally (roughly 40 MB):

In [4]:
latest_dataset.download(latest_dataset.name)

In [5]:
!ls
#!dir if on windows

GFS_CONUS_95km_20180109_1200.grib2 remote_access_example.ipynb
raw_download_example.ipynb         remote_subset_example.ipynb


<a name="open"></a>
We can also get a file-like object that lets us `read` from the file as if it were local:

In [6]:
fobj = latest_dataset.remote_open()
data = fobj.read()
print(len(data))

41530201


This is handy if you have Python code to read a particular format.

In [7]:
fobj

Just for fun, let's read the first 256 bytes of the file.

In [8]:
fobj.seek(0)

0

In [9]:
fobj.read(256)

b'\xae\x00\x00\x00\xf9\xaf\xc4\x87\xean\x1e\x1bJ.\x88\x0b\x83\xb1lzc\x1a\x00\x00\x19\xdfTZ\x00\x00\x00\x00\xcb\xb0\x02\x00\x00\x00\x80\x00\xc0\xf4\x0f\x03M\x00\x00\x00LTHA30 KWBC 091200 !grib2/ncep/SSIGFS/#213/201801091200F000/TMPK/300 hPa PRES-\x00\x00\x00chico.unidata.ucar.edu_v_idd.unidata.ucar.edu\x01\r\r\n344 \r\r\nLTHA30 KWBC 091200\r\r\nGRIB\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x1a?\x00\x00\x00\x15\x01\x00\x07\x00\x00\x02\x01\x01\x07\xe2\x01\t\x0c\x00\x00\x00\x01\x00\x00\x00A\x03\x00\x00\x00*\xd5\x00\x00\x00'

As you can see, you're dealing with the raw bytes of the file - no special interpretation of the file. In this case, we have a `GRIB2` file. Note that this file contains a collection of `GRIB2` records, and each record is prepended with an LDM metadata header. in this example, the LDM header is 

`LVHA20 KWBC 080600 !grib2/ncep/SSIGFS/#213/201801080600F000/VREL/200 hPa PRES,\x00\x00\x00leno.unidata.ucar.edu_v_idd.unidata.ucar.edu\x01\r\r\n802 \r\r\nLVHA20 KWBC 080600\r\r\n`

According to the `GRIB2` <a href="http://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_sect0.shtml">specification</a>, the first section of a `GRIB2` record, called SECTION 0 -- INDICATOR SECTION, is 16 octets long (an octet is 8 bytes), and begins with `GRIB`. So, the that sections looks like:

`GRIB\x00\x00\x00\x02`

The last part of the Indicator section, `\x02`, is an integer representing the version of the `GRIB` record. Just for fun, let's decode that last byte!

In [10]:
a = b'\x02'

In [11]:
int.from_bytes(b'\x02', byteorder='big')

2

So, we are dealing with a `GRIB` Edition 2 record. Congratulations - this is the first step of writing a `GRIB2` decoder in python!

But again, as you can see, the `.download()` and `remote_open()` methods are very basic, and themselves provide no "under the hood" support for easily reading the data into python.